Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [34]:
batch_size = 128
  
def predict(data, weights, biases):
    return tf.nn.softmax(tf.matmul(data, weights) + biases)

def reg_steps(start, end, mult):
    state = start
    while True:
        if state > end:
            break
        yield state
        state *= mult
    
#train logit with L2 regularization
def train(num_steps, reg_value):
  graph = tf.Graph()
  with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)   
      with tf.Session(graph=graph) as session:
        # Variables.
        weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))
        # Training computation
        logits = tf.matmul(tf_train_dataset, weights) + biases
        # Define regularization
        regularization = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + reg_value * regularization
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = predict(tf_valid_dataset, weights, biases)
        test_prediction = predict(tf_test_dataset, weights, biases)
        # start
        tf.initialize_all_variables().run()
        print("\nInitialized with regularization value %f" % reg_value)
        for step in xrange(num_steps):
          # Pick an offset within the training data, which has been randomized.
          # Note: we could use better randomization across epochs.
          offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
          # Generate a minibatch.
          batch_data = train_dataset[offset:(offset + batch_size), :]
          batch_labels = train_labels[offset:(offset + batch_size), :]
          # Prepare a dictionary telling the session where to feed the minibatch.
          # The key of the dictionary is the placeholder node of the graph to be fed,
          # and the value is the numpy array to feed to it.
          feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
          _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
          if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%\n" % accuracy(
              valid_prediction.eval(session=session), valid_labels))
        final_train_acc = accuracy(predictions, batch_labels)
        final_valid_acc =  accuracy(valid_prediction.eval(session=session), valid_labels)
        final_test_acc =  accuracy(test_prediction.eval(session=session), test_labels)
        return {
            "train_acc" : final_train_acc, 
            "valid_acc" : final_valid_acc,
            "test_acc"  : final_test_acc,
            "weights"   : weights,
            "biases"    : biases,
            "reg_value" : reg_value
        }


num_steps = 3001
reg_val_step = 3
reg_vals = list(reg_steps(1e-6, 0.1, 3))
results = map(lambda x: train(num_steps, x), reg_vals)
results.sort(key=lambda x: x["test_acc"], reverse=True)
opt_result = results[0]
worst_result = results[-1]

print("\nRange of regularization values: from %f to %f with step multiplier of %f" % (reg_vals[0], reg_vals[-1], reg_val_step))
print("Worst test accuracy %.3f for L2 value of %f" % (worst_result["test_acc"], worst_result["reg_value"]))
print("Optimal L2 regularization value was %f" % opt_result["reg_value"])
print("Optimal training accuracy %.3f" % opt_result["train_acc"])
print("Optimal validation accuracy %.3f" % opt_result["valid_acc"])
print("Optimal test accuracy %.3f" % opt_result["test_acc"])

final_weights = opt_result["weights"]
final_biases = opt_result["biases"]
final_reg = opt_result["reg_value"]


Initialized with regularization value 0.000001
Minibatch loss at step 0 : 14.5779
Minibatch accuracy: 10.2%
Validation accuracy: 12.1%

Minibatch loss at step 500 : 1.3953
Minibatch accuracy: 74.2%
Validation accuracy: 75.4%

Minibatch loss at step 1000 : 1.20619
Minibatch accuracy: 80.5%
Validation accuracy: 76.6%

Minibatch loss at step 1500 : 1.20891
Minibatch accuracy: 68.8%
Validation accuracy: 78.1%

Minibatch loss at step 2000 : 0.863969
Minibatch accuracy: 81.2%
Validation accuracy: 77.7%

Minibatch loss at step 2500 : 1.07416
Minibatch accuracy: 78.1%
Validation accuracy: 77.7%

Minibatch loss at step 3000 : 1.11129
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%


Initialized with regularization value 0.000003
Minibatch loss at step 0 : 15.5308
Minibatch accuracy: 16.4%
Validation accuracy: 12.2%

Minibatch loss at step 500 : 1.8088
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%

Minibatch loss at step 1000 : 1.2948
Minibatch accuracy: 82.0%
Validation accuracy: 7

In [ ]:
# Train neural net with L2 regularization

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
train(100)

Initialized
Minibatch loss at step 0 : 21.333
Minibatch accuracy: 7.8%
Validation accuracy: 12.2%
Test accuracy: 77.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [14]:
num_hidden = 1024
# Create a one-hidden-layer network with dropout
def one_layer_dropout():
  batch_size = 128
  graph = tf.Graph()
  with graph.as_default():
    # Define placeholders of data and dropout_prob
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    dropout_p = tf.placeholder(tf.float32)

    # Define weight and bias terms
    W_in = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
    b_in = tf.Variable(tf.zeros([num_hidden]))

    W_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    b_hidden = tf.Variable(tf.zeros([num_labels]))

    # Define computation and predictions
    # Train
    first_layer = tf.add(tf.matmul(train_X, W_in), b_in)
    relu_layer = tf.nn.relu(first_layer)
    dropout_layer = tf.nn.dropout(relu_layer, dropout_p)
    second_layer = tf.add(tf.matmul(dropout_layer, W_hidden), b_hidden)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(second_layer, train_y))
    train_prediction = tf.nn.softmax(second_layer)

    # Valid
    v_first_layer = tf.add(tf.matmul(valid_X, W_in), b_in)
    v_relu_layer = tf.nn.relu(v_first_layer)
    v_second_layer = tf.add(tf.matmul(v_relu_layer, W_hidden), b_hidden)
    valid_prediction = tf.nn.softmax(v_second_layer)

    # Test
    t_first_layer = tf.add(tf.matmul(test_X, W_in), b_in)
    t_relu_layer = tf.nn.relu(t_first_layer)
    t_second_layer = tf.add(tf.matmul(t_relu_layer, W_hidden), b_hidden)
    test_prediction = tf.nn.softmax(t_second_layer)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

  # Let's run it:

  num_steps = 3001

  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {train_X : batch_data, train_y : batch_labels, dropout_p : 0.3}
      _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step", step, ":", l)
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

one_layer_dropout()

Initialized
Minibatch loss at step 0 : 522.615
Minibatch accuracy: 13.3%
Validation accuracy: 14.7%
Minibatch loss at step 500 : 110.432
Minibatch accuracy: 69.5%
Validation accuracy: 80.9%
Minibatch loss at step 1000 : 57.6692
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 1500 : 80.3565
Minibatch accuracy: 70.3%
Validation accuracy: 82.8%
Minibatch loss at step 2000 : 53.8577
Minibatch accuracy: 71.9%
Validation accuracy: 83.4%
Minibatch loss at step 2500 : 49.2625
Minibatch accuracy: 74.2%
Validation accuracy: 83.6%
Minibatch loss at step 3000 : 38.9378
Minibatch accuracy: 77.3%
Validation accuracy: 83.7%
Test accuracy: 90.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [16]:
# Create a deep network with dropout
def deep_dropout():
  batch_size = 128
  n_hidden_1 = 2048
  n_hidden_2 = 2048

  graph = tf.Graph()
  with graph.as_default():
    # Define placeholders of data and dropout_prob
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    dropout_p1 = tf.placeholder(tf.float32)
    dropout_p2 = tf.placeholder(tf.float32)

    # Define weight and bias terms
    W_in = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1], stddev=0.03))
    b_in = tf.Variable(tf.zeros([n_hidden_1]))

    W_hidden_1 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.03))
    b_hidden_1 = tf.Variable(tf.zeros([n_hidden_2]))

    W_out = tf.Variable(tf.truncated_normal([n_hidden_2, num_labels], stddev=0.03))
    b_out = tf.Variable(tf.zeros([num_labels]))

    # Define computation and predictions
    # Train
    layer1 = tf.add(tf.matmul(train_X, W_in), b_in)
    relu1 = tf.nn.relu(layer1)
    dropout1 = tf.nn.dropout(relu1, dropout_p1)
    l2_loss_1 = tf.nn.l2_loss(W_in) + tf.nn.l2_loss(b_in)

    layer2 = tf.add(tf.matmul(dropout1, W_hidden_1), b_hidden_1)
    relu2 = tf.nn.relu(layer2)
    dropout2 = tf.nn.dropout(relu2, dropout_p2)
    l2_loss_2 = tf.nn.l2_loss(W_hidden_1) + tf.nn.l2_loss(b_hidden_1)

    out = tf.add(tf.matmul(dropout2, W_out), b_out)
    l2_loss_3 = tf.nn.l2_loss(W_out) + tf.nn.l2_loss(b_out)
    
    # Regularization
    regularization = l2_loss_1 + l2_loss_2 + l2_loss_3
    
    # Not adding regularization for now, but you can add it to loss as
    # loss = loss + 0.0001 * regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, train_y))
    train_prediction = tf.nn.softmax(out)

    # Valid
    v_layer1 = tf.add(tf.matmul(valid_X, W_in), b_in)
    v_relu1 = tf.nn.relu(v_layer1)

    v_layer2 = tf.add(tf.matmul(v_relu1, W_hidden_1), b_hidden_1)
    v_relu2 = tf.nn.relu(v_layer2)

    v_out = tf.add(tf.matmul(v_relu2, W_out), b_out)
    valid_prediction = tf.nn.softmax(v_out)

    # Test
    t_layer1 = tf.add(tf.matmul(test_X, W_in), b_in)
    t_relu1 = tf.nn.relu(t_layer1)

    t_layer2 = tf.add(tf.matmul(t_relu1, W_hidden_1), b_hidden_1)
    t_relu2 = tf.nn.relu(t_layer2)

    t_out = tf.add(tf.matmul(t_relu2, W_out), b_out)
    test_prediction = tf.nn.softmax(t_out)

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

  # Let's run it:

  num_steps = 12001

  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {train_X : batch_data, train_y : batch_labels,
                   dropout_p1 : 0.75, dropout_p2: 0.9}
      _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step", step, ":", l)
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

deep_dropout()

Initialized
Minibatch loss at step 0 : 2.30548
Minibatch accuracy: 14.8%
Validation accuracy: 26.8%
Minibatch loss at step 500 : 0.531638
Minibatch accuracy: 83.6%
Validation accuracy: 85.4%
Minibatch loss at step 1000 : 0.425914
Minibatch accuracy: 88.3%
Validation accuracy: 87.0%
Minibatch loss at step 1500 : 0.494915
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%
Minibatch loss at step 2000 : 0.285485
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Minibatch loss at step 2500 : 0.417747
Minibatch accuracy: 89.1%
Validation accuracy: 89.2%
Minibatch loss at step 3000 : 0.494398
Minibatch accuracy: 85.9%
Validation accuracy: 89.4%
Minibatch loss at step 3500 : 0.260349
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 4000 : 0.382056
Minibatch accuracy: 87.5%
Validation accuracy: 89.9%
Minibatch loss at step 4500 : 0.201449
Minibatch accuracy: 95.3%
Validation accuracy: 90.2%
Minibatch loss at step 5000 : 0.313237
Minibatch accuracy: 91.4%
Validati